In [6]:
import math
import matplotlib.pyplot as plt
from tensorflow import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
df=pd.read_csv("/home/sh4n1/miniproj/h1cpudata.csv")
df=df.drop(['Timestamp'],axis=1)
print(df.shape)
df.head(5)

(8634, 1)


,cpu_usage
0,10912.027692
1,10890.570362
2,10434.114431
3,10539.450415
4,10951.041020


In [7]:
print(df.iloc[0:, 3:4].values,df['cpu_usage'])


[] 0       10912.027692
1       10890.570362
2       10434.114431
3       10539.450415
4       10951.041020
            ...     
8629       83.878654
8630       81.927988
8631       76.075989
8632       85.829320
8633       83.878654
Name: cpu_usage, Length: 8634, dtype: float64


In [8]:
training_set = df.iloc[:6908, 3:4].values
test_set = df.iloc[6908:, 3:4].values

In [9]:
# Feature Scaling
sc = MinMaxScaler()
training_set_scaled = sc.fit_transform(df)
# Creating a data structure with 1 time-step and 1 output
X_train = []
y_train = []
for i in range(1, 6908):
    X_train.append(training_set_scaled[i-1:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
#(740, 60, 1)

In [10]:
model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
model.fit(X_train, y_train, epochs = 100, batch_size = 32)




2021-11-10 18:01:34.229672: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-10 18:01:34.230844: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-10 18:01:34.231209: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-11-10 18:01:34.231355: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-11-10 18:01:34.231500: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Epoch 1/100
216/216 [==============================] - 9s 13ms/step - loss: 0.0104
Epoch 2/100
216/216 [==============================] - 4s 16ms/step - loss: 0.0029
Epoch 3/100
216/216 [==============================] - 4s 16ms/step - loss: 0.0029
Epoch 4/100
216/216 [==============================] - 4s 16ms/step - loss: 0.0027
Epoch 5/100
216/216 [==============================] - 3s 16ms/step - loss: 0.0028
Epoch 6/100
216/216 [==============================] - 4s 17ms/step - loss: 0.0026
Epoch 7/100
216/216 [==============================] - 4s 17ms/step - loss: 0.0028
Epoch 8/100
216/216 [==============================] - 3s 16ms/step - loss: 0.0027
Epoch 9/100
216/216 [==============================] - 3s 16ms/step - loss: 0.0028
Epoch 10/100
216/216 [==============================] - 3s 16ms/step - loss: 0.0028
Epoch 11/100
216/216 [==============================] - 3s 16ms/step - loss: 0.0028
Epoch 12/100
216/216 [==============================] - 4s 16ms/step - loss: 0.0029
E

216/216 [==============================] - 4s 18ms/step - loss: 0.0025
Epoch 99/100
216/216 [==============================] - 4s 18ms/step - loss: 0.0026
Epoch 100/100
216/216 [==============================] - 4s 17ms/step - loss: 0.0026


In [20]:
X_train.shape

(6907, 1, 1)

In [12]:
dataset_train = df.iloc[:6908]
dataset_test = df.iloc[6908:]
dataset_total = pd.concat((dataset_train, dataset_test), axis = 1,ignore_index = True)
print(dataset_total)

inputs = dataset_total[len(dataset_total) - len(dataset_test) - 1:].values
print(inputs)
inputs.shape

inputs = inputs.reshape(-1,1)
print(inputs)

inputs = sc.transform(inputs)
print(inputs)

                 0          1
0     10912.027692        NaN
1     10890.570362        NaN
2     10434.114431        NaN
3     10539.450415        NaN
4     10951.041020        NaN
...            ...        ...
8629           NaN  83.878654
8630           NaN  81.927988
8631           NaN  76.075989
8632           NaN  85.829320
8633           NaN  83.878654

[8634 rows x 2 columns]
[[79.97732131         nan]
 [        nan 83.87865405]
 [        nan 68.27332307]
 ...
 [        nan 76.07598856]
 [        nan 85.82932043]
 [        nan 83.87865405]]
[[79.97732131]
 [        nan]
 [        nan]
 ...
 [85.82932043]
 [        nan]
 [83.87865405]]
[[0.00188292]
 [       nan]
 [       nan]
 ...
 [0.00239644]
 [       nan]
 [0.00222527]]


/home/sh4n1/.local/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [19]:
# Getting the predicted stock price of 2017
dataset_train = df.iloc[:6908,0]
dataset_test = df.iloc[6908:,0]
dataset_total = pd.concat((dataset_train, dataset_test), axis = 0,ignore_index = True)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 1:].values
#inputs = dataset_train.values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
print(inputs)
X_test = []
for i in range(6908,8634):
    X_test.append(inputs[i-1:i,0])
X_test = np.array(X_test)
print(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
print(X_test.shape)
# (459, 60, 1)

[[0.00188292]
 [0.00222527]
 [0.00085587]
 ...
 [0.00154057]
 [0.00239644]
 [0.00222527]]
[]
(1726, 0, 1)


/home/sh4n1/.local/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [17]:
#import torch
predicted_cpu_usage= model.predict(X_test)
predicted_cpu_usage = sc.inverse_transform(predicted_cpu_usage)

ValueError: in user code:

    /home/sh4n1/.local/lib/python3.9/site-packages/keras/engine/training.py:1586 predict_function  *
        return step_function(self, iterator)
    /home/sh4n1/.local/lib/python3.9/site-packages/keras/engine/training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/sh4n1/.local/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/sh4n1/.local/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/sh4n1/.local/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/sh4n1/.local/lib/python3.9/site-packages/keras/engine/training.py:1569 run_step  **
        outputs = model.pr